In [1]:
#!pip install torch_geometric
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GATConv

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cora - сеть научных статей по ml, где вершины — статьи, рёбра — ссылки (кто кого цитирует) у каждой вершины есть вектор признаков (bag-of-words по словам из текста статьи), а метка - научная категория статьи (класс).


In [5]:
dataset = Planetoid(root='data/Cora', name='Cora')
data = dataset[0].to(device)

print(dataset)
print("Число вершин:", data.num_nodes)
print("Число рёбер:", data.num_edges)
print("Число признаков:", dataset.num_features)
print("Число классов:", dataset.num_classes)

Processing...


Cora()
Число вершин: 2708
Число рёбер: 10556
Число признаков: 1433
Число классов: 7


Done!


Классификация вершин с помощью Graph Attention Network (GAT)

In [7]:
class GAT(torch.nn.Module):
    def __init__(
        self,
        in_channels: int,
        hidden_channels: int,
        out_channels: int,
        heads1: int = 8,
        heads2: int = 1,
        dropout: float = 0.6,
    ):
        super().__init__()
        self.dropout = dropout

        # первый слой: multi-head attention
        self.conv1 = GATConv(
            in_channels,
            hidden_channels,
            heads=heads1,
            dropout=dropout,
        )
        # второй слой тоже GAT но без concat (выход сразу нужной размерности классов)
        self.conv2 = GATConv(
            hidden_channels * heads1,
            out_channels,
            heads=heads2,
            concat=False,
            dropout=dropout,
        )

    def forward(self, x, edge_index):
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return x  # логиты классов


In [8]:

def train_one_epoch(model, data, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)

    # считаем loss только по train-вершинам
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

@torch.no_grad()
def eval_split(model, data, mask):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=-1)
    correct = (pred[mask] == data.y[mask]).sum().item()
    total = int(mask.sum())
    return correct / total

@torch.no_grad()
def eval_all(model, data):
    return {
        "train_acc": eval_split(model, data, data.train_mask),
        "val_acc":   eval_split(model, data, data.val_mask),
        "test_acc":  eval_split(model, data, data.test_mask),
    }


In [9]:
#цикл по разным learning rate

lrs = [0.005, 0.01, 0.02]  # можно поиграть
num_epochs = 200

best_val_acc = 0.0
best_stats = None
best_lr = None

for lr in lrs:
    print(f"\n=== Обучаем модель с lr = {lr} ===")
    model = GAT(
        in_channels=dataset.num_features,
        hidden_channels=8,
        out_channels=dataset.num_classes,
        heads1=8,
        heads2=1,
        dropout=0.6,
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)

    for epoch in range(1, num_epochs + 1):
        loss = train_one_epoch(model, data, optimizer)

        if epoch % 20 == 0 or epoch == 1:
            stats = eval_all(model, data)
            print(
                f"Epoch {epoch:03d} | loss={loss:.4f} | "
                f"train={stats['train_acc']:.3f} | "
                f"val={stats['val_acc']:.3f} | "
                f"test={stats['test_acc']:.3f}"
            )

    # финальная оценка для этого lr
    stats = eval_all(model, data)
    if stats["val_acc"] > best_val_acc:
        best_val_acc = stats["val_acc"]
        best_stats = stats
        best_lr = lr

print("\n=== Лучший результат по валидации ===")
print(f"Лучший lr: {best_lr}")
print(f"Train acc: {best_stats['train_acc']:.3f}")
print(f"Val   acc: {best_stats['val_acc']:.3f}")
print(f"Test  acc: {best_stats['test_acc']:.3f}")



=== Обучаем модель с lr = 0.005 ===
Epoch 001 | loss=1.9866 | train=0.521 | val=0.338 | test=0.359
Epoch 020 | loss=0.9181 | train=0.986 | val=0.792 | test=0.802
Epoch 040 | loss=0.6110 | train=0.993 | val=0.804 | test=0.812
Epoch 060 | loss=0.4926 | train=0.993 | val=0.796 | test=0.812
Epoch 080 | loss=0.5734 | train=1.000 | val=0.794 | test=0.803
Epoch 100 | loss=0.4129 | train=1.000 | val=0.796 | test=0.801
Epoch 120 | loss=0.5277 | train=1.000 | val=0.786 | test=0.798
Epoch 140 | loss=0.4012 | train=1.000 | val=0.794 | test=0.810
Epoch 160 | loss=0.3966 | train=1.000 | val=0.760 | test=0.786
Epoch 180 | loss=0.3802 | train=1.000 | val=0.786 | test=0.805
Epoch 200 | loss=0.4304 | train=1.000 | val=0.778 | test=0.798

=== Обучаем модель с lr = 0.01 ===
Epoch 001 | loss=2.0106 | train=0.579 | val=0.404 | test=0.398
Epoch 020 | loss=0.7303 | train=0.979 | val=0.774 | test=0.785
Epoch 040 | loss=0.5034 | train=1.000 | val=0.778 | test=0.797
Epoch 060 | loss=0.3879 | train=1.000 | val=0

GAT с lr =  0.01 обучилась хорошо, на трейне она почти идеально подогналась под данные,
а качество на валидации и тесте около 78–80%, что показывает хорошую обобщающую способность без сильного переобучения

Задача 2

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.datasets import RelLinkPredDataset
from torch_geometric.transforms import RandomLinkSplit

from torch_geometric.utils import subgraph
from torch_geometric.data import Data

FB15k-237- граф знаний, где вершины — сущности (люди, фильмы, компании и тд), а рёбра — типизированные отношения между ними (всего 237 разных типов отношений)

In [5]:
dataset = RelLinkPredDataset(root='data/FB15k237', name='FB15k-237')
data = dataset[0]

print(data)
print("Число вершин (entities):", data.num_nodes)
print("Число рёбер (трёхк):", data.edge_index.shape[1])
print("Пример edge_type shape:", data.edge_type.shape)
print("Максимальный тип ребра:", int(data.edge_type.max()), " всего отношений:",
      int(data.edge_type.max()) + 1)


Data(edge_index=[2, 544230], num_nodes=14541, edge_type=[544230], train_edge_index=[2, 272115], train_edge_type=[272115], valid_edge_index=[2, 17535], valid_edge_type=[17535], test_edge_index=[2, 20466], test_edge_type=[20466])
Число вершин (entities): 14541
Число рёбер (трёхк): 544230
Пример edge_type shape: torch.Size([544230])
Максимальный тип ребра: 473  всего отношений: 474


In [7]:
# для быстро сокращу немного датасет
max_edges = 40000

num_edges = data.edge_index.size(1)
perm = torch.randperm(num_edges)[:max_edges]

edge_index = data.edge_index[:, perm]
edge_type = data.edge_type[perm]

nodes = torch.unique(edge_index)
edge_index, edge_type = subgraph(
    nodes,
    edge_index,
    edge_attr=edge_type,  
    relabel_nodes=True,
)

data = Data(
    edge_index=edge_index,
    edge_type=edge_type,
    num_nodes=int(edge_index.max()) + 1,
)


In [8]:
# разбиение рёбер для задачи link prediction


transform = RandomLinkSplit(
    num_val=0.1,          #
    num_test=0.1,
    is_undirected=False,
    add_negative_train_samples=True,  # для генерации негативных примеров
    neg_sampling_ratio=1.0            # 1 отрицательное ребро на 1 положительное
)

train_data, val_data, test_data = transform(data)

train_data = train_data.to(device)
val_data   = val_data.to(device)
test_data  = test_data.to(device)

print(train_data)
print("train edge_label_index shape:", train_data.edge_label_index.shape)
print("train edge_label shape:", train_data.edge_label.shape)


Data(edge_index=[2, 32000], edge_type=[32000], num_nodes=12419, edge_label=[64000], edge_label_index=[2, 64000])
train edge_label_index shape: torch.Size([2, 64000])
train edge_label shape: torch.Size([64000])


In [9]:
from torch_geometric.nn import RGATConv

class RGATEncoder(nn.Module):
    def __init__(
        self,
        num_nodes: int,
        num_relations: int,
        emb_dim: int = 64,
        hidden_dim: int = 64,
        out_dim: int = 64,
        num_heads: int = 2,
        dropout: float = 0.3,
    ):
        super().__init__()
        self.dropout = dropout

        # лбучаемые эмбеддинги вершин
        self.node_emb = nn.Embedding(num_nodes, emb_dim)

        # attention по рёбрам с учётом типа связи
        self.conv1 = RGATConv(
            in_channels=emb_dim,
            out_channels=hidden_dim,
            num_relations=num_relations,
            heads=num_heads,
            dropout=dropout,
            concat=True,
        )

        self.conv2 = RGATConv(
            in_channels=hidden_dim * num_heads,
            out_channels=out_dim,
            num_relations=num_relations,
            heads=num_heads,
            dropout=dropout,
            concat=False,   # усредняются головы и получаем размер out_dim
        )

    def forward(self, edge_index, edge_type):
        x = self.node_emb.weight

        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv1(x, edge_index, edge_type)
        x = F.relu(x)

        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index, edge_type)

        return x  # размер [num_nodes, out_dim]

class LinkPredictor(nn.Module):
    def __init__(self, in_dim: int, hidden_dim: int = 64):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(2 * in_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),
        )

    def forward(self, x_i, x_j):
        # конкатенация эмбеддингов  + mlp
        h = torch.cat([x_i, x_j], dim=-1)
        out = self.mlp(h).squeeze(-1)  # логиты
        return out


In [10]:

def get_edge_embeddings(emb, edge_label_index):
    src, dst = edge_label_index
    return emb[src], emb[dst]


def train_one_epoch(encoder, predictor, data, optimizer):
    encoder.train()
    predictor.train()
    optimizer.zero_grad()

    # эмбеддинги всех узлов
    node_emb = encoder(data.edge_index, data.edge_type)
    ei, ej = get_edge_embeddings(node_emb, data.edge_label_index)
    logits = predictor(ei, ej)
    labels = data.edge_label.float()

    loss = F.binary_cross_entropy_with_logits(logits, labels)
    loss.backward()
    optimizer.step()

    return loss.item()


In [11]:
@torch.no_grad()
def eval_split(encoder, predictor, data):
    encoder.eval()
    predictor.eval()

    node_emb = encoder(data.edge_index, data.edge_type)
    ei, ej = get_edge_embeddings(node_emb, data.edge_label_index)
    logits = predictor(ei, ej)
    probs = torch.sigmoid(logits)

    labels = data.edge_label.float()

    pred_labels = (probs > 0.5).float()
    acc = (pred_labels == labels).float().mean().item()

    return acc


In [12]:
num_nodes = data.num_nodes
num_relations = int(data.edge_type.max().item()) + 1

lrs = [1e-3, 5e-4]
num_epochs = 20

best_val_acc = 0.0
best_test_acc = 0.0
best_lr = None

for lr in lrs:
    print(f"\n=== Обучаем RGAT-модель для link prediction, lr = {lr} ===")

    encoder = RGATEncoder(
        num_nodes=num_nodes,
        num_relations=num_relations,
        emb_dim=64,
        hidden_dim=64,
        out_dim=64,
        num_heads=2,
        dropout=0.3,
    ).to(device)

    predictor = LinkPredictor(in_dim=64, hidden_dim=64).to(device)

    optimizer = torch.optim.Adam(
        list(encoder.parameters()) + list(predictor.parameters()),
        lr=lr,
        weight_decay=1e-5,
    )

    for epoch in range(1, num_epochs + 1):
        loss = train_one_epoch(encoder, predictor, train_data, optimizer)

        if epoch % 10 == 0 or epoch == 1:
            train_acc = eval_split(encoder, predictor, train_data)
            val_acc = eval_split(encoder, predictor, val_data)
            test_acc = eval_split(encoder, predictor, test_data)

            print(
                f"Epoch {epoch:03d} | loss={loss:.4f} | "
                f"train_acc={train_acc:.3f} | val_acc={val_acc:.3f} | test_acc={test_acc:.3f}"
            )

    val_acc = eval_split(encoder, predictor, val_data)
    test_acc = eval_split(encoder, predictor, test_data)

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_test_acc = test_acc
        best_lr = lr

print("\n=== Лучший результат по валидации для RGATLinkPrediction ===")
print(f"Лучший lr: {best_lr}")
print(f"Val   acc: {best_val_acc:.3f}")
print(f"Test  acc: {best_test_acc:.3f}")



=== Обучаем RGAT-модель для link prediction, lr = 0.001 ===
Epoch 001 | loss=0.6937 | train_acc=0.506 | val_acc=0.508 | test_acc=0.502
Epoch 010 | loss=0.6155 | train_acc=0.709 | val_acc=0.700 | test_acc=0.686
Epoch 020 | loss=0.5578 | train_acc=0.754 | val_acc=0.738 | test_acc=0.733

=== Обучаем RGAT-модель для link prediction, lr = 0.0005 ===
Epoch 001 | loss=0.6971 | train_acc=0.501 | val_acc=0.500 | test_acc=0.501
Epoch 010 | loss=0.6769 | train_acc=0.544 | val_acc=0.535 | test_acc=0.534
Epoch 020 | loss=0.6289 | train_acc=0.673 | val_acc=0.661 | test_acc=0.659

=== Лучший результат по валидации для RGATLinkPrediction ===
Лучший lr: 0.001
Val   acc: 0.738
Test  acc: 0.733


RGAT-модель хорошо выучила структуру гетерогенного графа, качество на валидации и тесте около 0.73, без сильного переобучения